# Aerial-Cactus-Identification
Determine whether an image contains a columnar cactus

### Load and preprocessing data

In [58]:
import pandas as pd
import os
import random
from sklearn.utils import shuffle

In [7]:
ls aerial-cactus-identification_data/

sample_submission.csv* train/
test/                  train.csv*


In [92]:
df = pd.read_csv("./aerial-cactus-identification_data/train.csv")
df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [93]:
df['id'] = df['id'].apply(lambda x:os.path.join('./aerial-cactus-identification_data/train', x))

In [94]:
df = shuffle(df, random_state=10)
df.head()

,id,has_cactus
1301,./aerial-cactus-identification_data/train/1296...,1
13951,./aerial-cactus-identification_data/train/cb03...,1
12534,./aerial-cactus-identification_data/train/b5e9...,1
2128,./aerial-cactus-identification_data/train/1dc7...,1
3222,./aerial-cactus-identification_data/train/2d4c...,1


In [96]:
df.reset_index(inplace=True)

In [97]:
df

,index,id,has_cactus
0,0,./aerial-cactus-identification_data/train/0004...,1
1,1,./aerial-cactus-identification_data/train/000c...,1
2,2,./aerial-cactus-identification_data/train/000d...,1
3,3,./aerial-cactus-identification_data/train/0011...,1
4,4,./aerial-cactus-identification_data/train/0014...,1
5,5,./aerial-cactus-identification_data/train/0017...,1
6,6,./aerial-cactus-identification_data/train/0021...,0
7,7,./aerial-cactus-identification_data/train/0024...,0
8,8,./aerial-cactus-identification_data/train/0029...,1
9,9,./aerial-cactus-identification_data/train/0035...,1
